In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
from matplotlib.colors import Normalize
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import mpl_toolkits.axes_grid1
import sys
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
from matplotlib.ticker import MultipleLocator

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf
import functions_angular_crossings as fac


### Reading hdf5 data

In [ ]:
bh_r=5.43, # km
bh_x=48.0, # km
bh_y=48.0, # km
bh_z=16.0, # km
# bh_x=0.0, # km
# bh_y=0.0, # km
# bh_z=0.0, # km

In [ ]:
N_dot_nu_e_eq_h5 = h5py.File('/home/erick/software/devscrpts/gw170817_paper_plots/N_dot_nu_e_eq.h5', 'r')
xlow_nu_e = N_dot_nu_e_eq_h5['xlow'][:]
ylow_nu_e = N_dot_nu_e_eq_h5['ylow'][:]
N_dot_nu_e_eq_log10_low = N_dot_nu_e_eq_h5['N_dot_nu_e_eq_log10_low'][:]
xup_nu_e = N_dot_nu_e_eq_h5['xup'][:]
yup_nu_e = N_dot_nu_e_eq_h5['yup'][:]
N_dot_nu_e_eq_log10_up = N_dot_nu_e_eq_h5['N_dot_nu_e_eq_log10_up'][:]
N_dot_nu_e_eq_h5.close()

N_dot_nubar_e_eq_h5 = h5py.File('/home/erick/software/devscrpts/gw170817_paper_plots/N_dot_nubar_e_eq.h5', 'r')
xlow_nubar_e = N_dot_nubar_e_eq_h5['xlow'][:]
ylow_nubar_e = N_dot_nubar_e_eq_h5['ylow'][:]
N_dot_nubar_e_eq_log10_low = N_dot_nubar_e_eq_h5['N_dot_nubar_e_eq_log10_low'][:]
xup_nubar_e = N_dot_nubar_e_eq_h5['xup'][:]
yup_nubar_e = N_dot_nubar_e_eq_h5['yup'][:]
N_dot_nubar_e_eq_log10_up = N_dot_nubar_e_eq_h5['N_dot_nubar_e_eq_log10_up'][:]

N_dot_nu_x_eq_h5 = h5py.File('/home/erick/software/devscrpts/gw170817_paper_plots/N_dot_nu_x_eq.h5', 'r')
xlow_nu_x = N_dot_nu_x_eq_h5['xlow'][:]
ylow_nu_x = N_dot_nu_x_eq_h5['ylow'][:]
N_dot_nu_x_eq_log10_low = N_dot_nu_x_eq_h5['N_dot_nu_x_eq_log10_low'][:]
xup_nu_x = N_dot_nu_x_eq_h5['xup'][:]
yup_nu_x = N_dot_nu_x_eq_h5['yup'][:]
N_dot_nu_x_eq_log10_up = N_dot_nu_x_eq_h5['N_dot_nu_x_eq_log10_up'][:]
N_dot_nu_x_eq_h5.close()

one_km_in_cm = 1e5 # cm

n_dot_nu_e_eq_log10_low = np.log10((10**(N_dot_nu_e_eq_log10_low))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nu_e_eq_log10_up = np.log10((10**(N_dot_nu_e_eq_log10_up))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nubar_e_eq_log10_low = np.log10((10**(N_dot_nubar_e_eq_log10_low))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nubar_e_eq_log10_up = np.log10((10**(N_dot_nubar_e_eq_log10_up))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nu_x_eq_log10_low = np.log10((10**(N_dot_nu_x_eq_log10_low))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nu_x_eq_log10_up = np.log10((10**(N_dot_nu_x_eq_log10_up))/one_km_in_cm**3) # neutrino / ( s cm3 )

T_map = 'hot'
ye_map = 'hot'
rho_map = 'hot'

# color bar limits
n_dot_max = np.nanmax([
    np.nanmax(n_dot_nu_e_eq_log10_low),
    np.nanmax(n_dot_nu_e_eq_log10_up),
    np.nanmax(n_dot_nubar_e_eq_log10_low),
    np.nanmax(n_dot_nubar_e_eq_log10_up),
    np.nanmax(n_dot_nu_x_eq_log10_low),
    np.nanmax(n_dot_nu_x_eq_log10_up)
])
n_dot_min = np.nanmin([
    np.nanmin(n_dot_nu_e_eq_log10_low),
    np.nanmin(n_dot_nu_e_eq_log10_up),
    np.nanmin(n_dot_nubar_e_eq_log10_low),
    np.nanmin(n_dot_nubar_e_eq_log10_up),
    np.nanmin(n_dot_nu_x_eq_log10_low),
    np.nanmin(n_dot_nu_x_eq_log10_up)
])

print(f"n_dot_min = {n_dot_min}")
print(f"n_dot_max = {n_dot_max}")
# n_dot_max = 37
n_dot_min = 32

# # Set NaN for points inside the black hole (r < bh_r) in both slices
# r_bh = bh_r  # km

# # For z = const (up) slice
# r_up = np.sqrt((xup - bh_x)**2 + (yup - bh_y)**2 + (z[z_slice_indx] - bh_z)**2)
# mask_up = r_up < r_bh
# temp_slice_up[mask_up] = np.nan
# ye_slice_up[mask_up] = np.nan
# rho_slice_up[mask_up] = np.nan

# # For y = const (low) slice
# r_low = np.sqrt((xlow - bh_x)**2 + (y[y_slice_indx] - bh_y)**2 + (ylow - bh_z)**2)
# mask_low = r_low < r_bh
# temp_slice_low[mask_low] = np.nan
# ye_slice_low[mask_low] = np.nan
# rho_slice_low[mask_low] = np.nan

xmid = 48 # km
ymid = 48 # km
zmid = 16 # km

# create 2x3 figure
fig, axes = plt.subplots(2, 3, figsize=(18, 12), sharex='col', sharey='row')
plt.subplots_adjust(wspace=0, hspace=-0.255)
fig.align_labels()

# # colorbars
# norm = mpl.colors.Normalize(vmin=temp_min, vmax=temp_max)
# sm = mpl.cm.ScalarMappable(cmap=T_map, norm=norm)
# sm.set_array([])  # Required for ScalarMappable even if unused
# divider = mpl_toolkits.axes_grid1.make_axes_locatable(axes[0,0])
# cax = divider.append_axes("top", size="4%", pad=0.05)
# cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
# cbar.ax.tick_params(which="both",direction="in")
# cbar.ax.xaxis.set_ticks_position('top')
# cbar.ax.xaxis.set_label_position('top')
# cbar.set_label(r"$T\, [\mathrm{MeV}]$", labelpad=10)
# cax.xaxis.set_major_locator(MultipleLocator(1))

# norm = mpl.colors.Normalize(vmin=ye_min, vmax=ye_max)
# sm = mpl.cm.ScalarMappable(cmap=ye_map, norm=norm)
# sm.set_array([])  # Required for ScalarMappable even if unused
# divider = mpl_toolkits.axes_grid1.make_axes_locatable(axes[0,1])
# cax = divider.append_axes("top", size="4%", pad=0.05)
# cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
# cbar.ax.tick_params(which="both",direction="in")
# cbar.ax.xaxis.set_ticks_position('top')
# cbar.ax.xaxis.set_label_position('top')
# cbar.set_label(r"$Y_e$", labelpad=10)

# norm = mpl.colors.Normalize(vmin=rho_min, vmax=rho_max)
# sm = mpl.cm.ScalarMappable(cmap=rho_map, norm=norm)
# sm.set_array([])  # Required for ScalarMappable even if unused
# divider = mpl_toolkits.axes_grid1.make_axes_locatable(axes[0,2])
# cax = divider.append_axes("top", size="4%", pad=0.05)
# cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
# cbar.ax.tick_params(which="both",direction="in")
# cbar.ax.xaxis.set_ticks_position('top')
# cbar.ax.xaxis.set_label_position('top')
# cbar.set_label(r"$\log\ \rho \ [\mathrm{g/ccm}]$", labelpad=10)
# cax.xaxis.set_major_locator(MultipleLocator(1))


# Shared colorbar
# Shared colorbar (manually set position)

norm = Normalize(n_dot_min, n_dot_max)
sm = plt.cm.ScalarMappable(cmap=T_map, norm=norm)
sm.set_array([])

# Manually set colorbar position using fig.add_axes([left, bottom, width, height])
# Example: right side, full height, narrow bar
cbar_ax = fig.add_axes([0.91, 0.15, 0.02, 0.64])  # adjust as needed
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label(r'$\log\ \dot{n} \, [\mathrm{s}^{-1}\,\mathrm{cm}^{-3}]$', fontsize=28)
cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())


# set axis labels
for ax in axes[-1,:]:
    ax.set_xlabel(r'$x\ [\mathrm{km}]$', fontsize=28)
axes[0,0].set_ylabel(r'$z\ [\mathrm{km}]$', fontsize=28)
axes[1,0].set_ylabel(r'$y\ [\mathrm{km}]$', fontsize=28)
# remove xtick labels on all but the bottom row
for ax in axes[:-1,:].flatten():
    for label in ax.get_xticklabels():
        label.set_visible(False)
for ax in axes[:,1:].flatten():
    for label in ax.get_yticklabels():
        label.set_visible(False)
for ax in axes.flat:
    ax.set_aspect('equal')
    ax.xaxis.set_major_locator(MultipleLocator(20))
    ax.yaxis.set_major_locator(MultipleLocator(20))
    ax.minorticks_on()
    ax.tick_params(axis='both', which='major', labelsize=28)
    # apply_custom_settings(ax, False)
# mask out the black hole region
# Set white as the color for NaN values in the colormap
# cmap_with_nan = mpl.cm.get_cmap(cmap).copy()
# cmap_with_nan.set_bad(color='white')

# Temperature, z=const slice (bottom left)
axes[1,0].pcolormesh(xup_nu_e-xmid , yup_nu_e-ymid , n_dot_nu_e_eq_log10_up , shading='auto', cmap=T_map, vmin=n_dot_min, vmax=n_dot_max)
circle_bh = plt.Circle((bh_x[0] - xmid, bh_y[0] - ymid), bh_r[0], color='k', fill=False, linestyle='--', linewidth=1)
axes[1,0].add_patch(circle_bh)
legend_circle = Line2D([0], [0], color='k', linestyle='dashed', linewidth=1, label='Black hole')
axes[1,0].legend(handles=[legend_circle], loc='upper right', frameon=False, fontsize=22)

# Temperature, y=const slice (top left)
axes[0,0].pcolormesh(xlow_nu_e-xmid, ylow_nu_e-zmid, n_dot_nu_e_eq_log10_low, shading='auto', cmap=T_map, vmin=n_dot_min, vmax=n_dot_max)
circle_bh_00 = plt.Circle((bh_x[0] - xmid, bh_z[0] - zmid), bh_r[0], color='k', fill=False, linestyle='--', linewidth=1)
axes[0,0].add_patch(circle_bh_00)

# Ye, z=const slice (bottom middle)
axes[1,1].pcolormesh(xup_nubar_e-xmid , yup_nubar_e-ymid , n_dot_nubar_e_eq_log10_up , shading='auto', cmap=ye_map, vmin=n_dot_min, vmax=n_dot_max)
circle_bh_11 = plt.Circle((bh_x[0] - xmid, bh_y[0] - ymid), bh_r[0], color='k', fill=False, linestyle='--', linewidth=1)
axes[1,1].add_patch(circle_bh_11)

# Ye, y=const slice (top middle)
axes[0,1].pcolormesh(xlow_nubar_e-xmid, ylow_nubar_e-zmid, n_dot_nubar_e_eq_log10_low, shading='auto', cmap=ye_map, vmin=n_dot_min, vmax=n_dot_max)
circle_bh_01 = plt.Circle((bh_x[0] - xmid, bh_z[0] - zmid), bh_r[0], color='k', fill=False, linestyle='--', linewidth=1)
axes[0,1].add_patch(circle_bh_01)

# log10(rho), z=const slice (bottom right)
axes[1,2].pcolormesh(xup_nu_x-xmid , yup_nu_x-ymid , n_dot_nu_x_eq_log10_up , shading='auto', cmap=rho_map, vmin=n_dot_min  , vmax=n_dot_max  )
circle_bh_12 = plt.Circle((bh_x[0] - xmid, bh_y[0] - ymid), bh_r[0], color='k', fill=False, linestyle='--', linewidth=1)
axes[1,2].add_patch(circle_bh_12)

# log10(rho), y=const slice (top right)
axes[0,2].pcolormesh(xlow_nu_x-xmid, ylow_nu_x-zmid, n_dot_nu_x_eq_log10_low, shading='auto', cmap=rho_map, vmin=n_dot_min, vmax=n_dot_max)
circle_bh_02 = plt.Circle((bh_x[0] - xmid, bh_z[0] - zmid), bh_r[0], color='k', fill=False, linestyle='--', linewidth=1)
axes[0,2].add_patch(circle_bh_02)

axes[1,0].text(0.1, 0.15, r'$\nu_e$', transform=axes[1,0].transAxes, fontsize=28,
    verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='None', alpha=0.7, edgecolor='none'))
# axes[0,0].text(0.1, 0.15, r'$\nu_e$', transform=axes[0,0].transAxes, fontsize=28,
    # verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='None', alpha=0.7, edgecolor='none'))

axes[1,1].text(0.1, 0.15, r'$\bar{\nu}_e$', transform=axes[1,1].transAxes, fontsize=28,
    verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='None', alpha=0.7, edgecolor='none'))
# axes[0,1].text(0.1, 0.15, r'$\bar{\nu}_e$', transform=axes[0,1].transAxes, fontsize=28,
#     verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='None', alpha=0.7, edgecolor='none'))

axes[1,2].text(0.1, 0.15, r'$\nu_x$', transform=axes[1,2].transAxes, fontsize=28,
    verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='None', alpha=0.7, edgecolor='none'))
# axes[0,2].text(0.1, 0.15, r'$\nu_x$', transform=axes[0,2].transAxes, fontsize=28,
#     verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='None', alpha=0.7, edgecolor='none'))

plt.savefig('plots/n_dot.png', bbox_inches='tight')

In [ ]:
N_dot_nu_e_eq_h5 = h5py.File('/home/erick/software/devscrpts/gw170817_paper_plots/N_dot_nu_e_eq.h5', 'r')
xlow_nu_e = N_dot_nu_e_eq_h5['xlow'][:]
ylow_nu_e = N_dot_nu_e_eq_h5['ylow'][:]
N_dot_nu_e_eq_log10_low = N_dot_nu_e_eq_h5['N_dot_nu_e_eq_log10_low'][:]
xup_nu_e = N_dot_nu_e_eq_h5['xup'][:]
yup_nu_e = N_dot_nu_e_eq_h5['yup'][:]
N_dot_nu_e_eq_log10_up = N_dot_nu_e_eq_h5['N_dot_nu_e_eq_log10_up'][:]
N_dot_nu_e_eq_h5.close()

N_dot_nubar_e_eq_h5 = h5py.File('/home/erick/software/devscrpts/gw170817_paper_plots/N_dot_nubar_e_eq.h5', 'r')
xlow_nubar_e = N_dot_nubar_e_eq_h5['xlow'][:]
ylow_nubar_e = N_dot_nubar_e_eq_h5['ylow'][:]
N_dot_nubar_e_eq_log10_low = N_dot_nubar_e_eq_h5['N_dot_nubar_e_eq_log10_low'][:]
xup_nubar_e = N_dot_nubar_e_eq_h5['xup'][:]
yup_nubar_e = N_dot_nubar_e_eq_h5['yup'][:]
N_dot_nubar_e_eq_log10_up = N_dot_nubar_e_eq_h5['N_dot_nubar_e_eq_log10_up'][:]

N_dot_nu_x_eq_h5 = h5py.File('/home/erick/software/devscrpts/gw170817_paper_plots/N_dot_nu_x_eq.h5', 'r')
xlow_nu_x = N_dot_nu_x_eq_h5['xlow'][:]
ylow_nu_x = N_dot_nu_x_eq_h5['ylow'][:]
N_dot_nu_x_eq_log10_low = N_dot_nu_x_eq_h5['N_dot_nu_x_eq_log10_low'][:]
xup_nu_x = N_dot_nu_x_eq_h5['xup'][:]
yup_nu_x = N_dot_nu_x_eq_h5['yup'][:]
N_dot_nu_x_eq_log10_up = N_dot_nu_x_eq_h5['N_dot_nu_x_eq_log10_up'][:]
N_dot_nu_x_eq_h5.close()

N_dot_nu_e_eq_over_N_dot_nubar_e_eq_up = 10**(N_dot_nu_e_eq_log10_up) / 10**(N_dot_nubar_e_eq_log10_up)
N_dot_nu_e_eq_over_N_dot_nubar_e_eq_low = 10**(N_dot_nu_e_eq_log10_low) / 10**(N_dot_nubar_e_eq_log10_low)

log10_n_dot_nu_e_eq_over_N_dot_nubar_e_eq_up = np.log10(N_dot_nu_e_eq_over_N_dot_nubar_e_eq_up)   # dimensionaless
log10_n_dot_nu_e_eq_over_N_dot_nubar_e_eq_low = np.log10(N_dot_nu_e_eq_over_N_dot_nubar_e_eq_low) # dimensionaless  

one_km_in_cm = 1e5 # cm

n_dot_nu_e_eq_log10_low = np.log10((10**(N_dot_nu_e_eq_log10_low))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nu_e_eq_log10_up = np.log10((10**(N_dot_nu_e_eq_log10_up))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nubar_e_eq_log10_low = np.log10((10**(N_dot_nubar_e_eq_log10_low))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nubar_e_eq_log10_up = np.log10((10**(N_dot_nubar_e_eq_log10_up))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nu_x_eq_log10_low = np.log10((10**(N_dot_nu_x_eq_log10_low))/one_km_in_cm**3) # neutrino / ( s cm3 )
n_dot_nu_x_eq_log10_up = np.log10((10**(N_dot_nu_x_eq_log10_up))/one_km_in_cm**3) # neutrino / ( s cm3 )

ndot_map = 'hot'
ndot_ratio_map = 'PuOr'

# color bar limits
n_dot_max = np.nanmax([
    np.nanmax(n_dot_nu_e_eq_log10_low),
    np.nanmax(n_dot_nu_e_eq_log10_up),
    np.nanmax(n_dot_nubar_e_eq_log10_low),
    np.nanmax(n_dot_nubar_e_eq_log10_up),
    np.nanmax(n_dot_nu_x_eq_log10_low),
    np.nanmax(n_dot_nu_x_eq_log10_up)
])
n_dot_min = np.nanmin([
    np.nanmin(n_dot_nu_e_eq_log10_low),
    np.nanmin(n_dot_nu_e_eq_log10_up),
    np.nanmin(n_dot_nubar_e_eq_log10_low),
    np.nanmin(n_dot_nubar_e_eq_log10_up),
    np.nanmin(n_dot_nu_x_eq_log10_low),
    np.nanmin(n_dot_nu_x_eq_log10_up)
])

print(f"n_dot_min = {n_dot_min}")
print(f"n_dot_max = {n_dot_max}")
# n_dot_max = 37
n_dot_min = 32

ratio_limits_min = -1.1
ratio_limits_max = +1.1

xmid = 96 / 2 # km
ymid = 96 / 2
zmid = 32 / 2 # km

#############################################################################
# create figure

fig, axes = plt.subplots(2, 4, figsize=(24, 12), sharex='col', sharey='row')
plt.subplots_adjust(wspace=0, hspace=-0.256)
fig.align_labels()

cmap_1 = ndot_map
norm = mpl.colors.Normalize(vmin=n_dot_min, vmax=n_dot_max)
sm = mpl.cm.ScalarMappable(cmap=cmap_1, norm=norm)
sm.set_array([])
bbox_left = axes[0,0].get_position()
bbox_right = axes[0,2].get_position()
cax_height = 0.02
cax_pad = 0.03
cax = fig.add_axes([bbox_left.x0,
                    bbox_left.y1*0.88 + cax_pad,
                    bbox_right.x1 - bbox_left.x0,
                    cax_height])
cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
cbar.ax.tick_params(which="both", direction="in", labelsize=28)
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.set_minor_locator(AutoMinorLocator())
cbar.ax.xaxis.set_major_locator(MultipleLocator(1))
cbar.set_label(r'$\log_{10}\ \dot{n} \, [\mathrm{s}^{-1}\,\mathrm{cm}^{-3}]$', labelpad=10, fontsize=28)

cmap_2 = ndot_ratio_map
instability_norm = mpl.colors.Normalize(vmin=ratio_limits_min, vmax=ratio_limits_max)
instability_sm = mpl.cm.ScalarMappable(cmap=cmap_2, norm=instability_norm)
instability_sm.set_array([])
bbox_left = axes[0,3].get_position()
bbox_right = axes[0,3].get_position()
cax_height = 0.02
cax_pad = 0.03
cax = fig.add_axes([bbox_left.x0,
                    bbox_left.y1 * 0.88 + cax_pad,
                    bbox_right.x1 - bbox_left.x0,
                    cax_height])
cbar = plt.colorbar(instability_sm, cax=cax, orientation='horizontal')
cbar.ax.tick_params(which="both", direction="in", labelsize=28)
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.set_minor_locator(AutoMinorLocator())
cbar.ax.xaxis.set_major_locator(MultipleLocator(1))
cbar.set_label(r'$ \log_{10}\ \dot{n}_{\nu_e} / \dot{n}_{\bar{\nu}_e} $', labelpad=10, fontsize=28)

# set axis labels
for ax in axes[-1,:]:
    ax.set_xlabel(r'$x\,[\mathrm{km}]$', fontsize=28)
axes[0,0].set_ylabel(r'$z\,[\mathrm{km}]$', fontsize=28)
axes[1,0].set_ylabel(r'$y\,[\mathrm{km}]$', fontsize=28)
# remove xtick labels on all but the bottom row
for ax in axes[:-1,:].flatten():
    for label in ax.get_xticklabels():
        label.set_visible(False) 
for ax in axes[:,1:].flatten():
    for label in ax.get_yticklabels():
        label.set_visible(False)

for ax in axes.flat:
    ax.set_aspect('equal')
    ax.xaxis.set_major_locator(MultipleLocator(20))
    ax.yaxis.set_major_locator(MultipleLocator(20))
    ax.minorticks_on()
    ax.tick_params(axis='both', labelsize=28)
    # pf.apply_custom_settings(ax, False)

# electron neutrinos emission rate
axes[0,0].pcolormesh(xlow_nu_e-xmid, ylow_nu_e-zmid, n_dot_nu_e_eq_log10_low, shading='auto', cmap=cmap_1, vmin=n_dot_min, vmax=n_dot_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_z[0]-zmid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[0,0].add_patch(circle_bh)
axes[1,0].pcolormesh(xup_nu_e-xmid , yup_nu_e-ymid , n_dot_nu_e_eq_log10_up , shading='auto', cmap=cmap_1, vmin=n_dot_min, vmax=n_dot_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_y[0]-ymid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[1,0].add_patch(circle_bh)
legend_circle = Line2D([0], [0], color='k', linestyle='dashed', linewidth=1, label='Black hole')
axes[1,0].legend(handles=[legend_circle], loc='upper right', frameon=False, fontsize=24)
axes[0,0].text(0.5, 0.8, r'$\nu_e$', transform=axes[0,0].transAxes, fontsize=40, ha='left', va='bottom', color='white')

# electron antineutrinos emission rate
axes[0,1].pcolormesh(xlow_nubar_e-xmid, ylow_nubar_e-zmid, n_dot_nubar_e_eq_log10_low, shading='auto', cmap=cmap_1, vmin=n_dot_min  , vmax=n_dot_max  )
circle_bh = plt.Circle((bh_x[0]-xmid, bh_z[0]-zmid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[0,1].add_patch(circle_bh)
axes[1,1].pcolormesh(xup_nubar_e-xmid , yup_nubar_e-ymid , n_dot_nubar_e_eq_log10_up   , shading='auto', cmap=cmap_1, vmin=n_dot_min  , vmax=n_dot_max  )
circle_bh = plt.Circle((bh_x[0]-xmid, bh_y[0]-ymid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[1,1].add_patch(circle_bh)
axes[0,1].text(0.5, 0.8, r'$\bar{\nu}_e$', transform=axes[0,1].transAxes, fontsize=40, ha='left', va='bottom', color='white')

# heavy lepton neutrinos emission rate
axes[0,2].pcolormesh(xlow_nu_x-xmid, ylow_nu_x-zmid, n_dot_nu_x_eq_log10_low, shading='auto', cmap=cmap_1, vmin=n_dot_min, vmax=n_dot_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_z[0]-zmid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[0,2].add_patch(circle_bh)
axes[1,2].pcolormesh(xup_nu_x-xmid , yup_nu_x-ymid , n_dot_nu_x_eq_log10_up, shading='auto', cmap=cmap_1, vmin=n_dot_min, vmax=n_dot_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_y[0]-ymid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[1,2].add_patch(circle_bh)
axes[0,2].text(0.5, 0.8, r'$\nu_x$', transform=axes[0,2].transAxes, fontsize=40, ha='left', va='bottom', color='white')

# ration of electron neutrinos to electron antineutrinos emission rates
axes[0,3].pcolormesh(xlow_nu_x-xmid, ylow_nu_x-zmid, log10_n_dot_nu_e_eq_over_N_dot_nubar_e_eq_low  , shading='auto', cmap=cmap_2, vmin=ratio_limits_min  , vmax=ratio_limits_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_z[0]-zmid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[0,3].add_patch(circle_bh)
axes[1,3].pcolormesh(xup_nu_x-xmid , yup_nu_x-ymid , log10_n_dot_nu_e_eq_over_N_dot_nubar_e_eq_up   , shading='auto', cmap=cmap_2, vmin=ratio_limits_min  , vmax=ratio_limits_max)
circle_bh = plt.Circle((bh_x[0]-xmid, bh_y[0]-ymid), bh_r[0], color='k', fill=False, linestyle='dashed', linewidth=1)
axes[1,3].add_patch(circle_bh)
axes[0,3].text(0.4, 0.8, r'$\nu_e / \bar{\nu}_e$', transform=axes[0,3].transAxes, fontsize=40, ha='left', va='bottom', color='white')

plt.savefig('plots/n_dot_and_ration_d_dot.png', bbox_inches='tight')